### <center> Pandas test </center>
Прочитайте данные из файла **`pandas_test.csv`** 

Для каждого уникального **`id`** оставьте только одну строку с самой поздней датой. 

Для полученной таблицы подсчитайте:

    а) сколько в каждом городе было уникальных id;
    b) сколько id содержится в каждом множестве городов.

Порядок вывода не важен.

Пример:

city, id

"Moscow, Sochi", 1

"Sochi, Moscow", 2

"Moscow", 3

    a) Sochi - 2
       Moscow - 3

    b) Sochi, Moscow - 2
       Moscow - 1

In [1]:
!python --version

Python 3.6.6


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('pandas_test.csv')

In [4]:
df.shape

(1000, 3)

In [5]:
df.head(7)

,city,date,id
0,"Kazan, St-Petersburg, Sochi",2014-10-06 03:16:18,49
1,"Moscow, Kazan",2013-01-04 09:44:04,252
2,"Kazan, Moscow",2013-12-04 11:39:03,208
3,"St-Petersburg, Kazan",2014-09-04 10:09:53,81
4,"St-Petersburg, Moscow, Sochi",2014-09-25 02:04:14,109
5,"Moscow, Sochi, St-Petersburg, Kazan",2014-02-24 09:30:03,27
6,"Moscow, St-Petersburg, Sochi, Kazan",2014-09-20 18:31:50,3


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
city    805 non-null object
date    1000 non-null object
id      1000 non-null int64
dtypes: int64(1), object(2)
memory usage: 23.5+ KB


Сразу замечаем, что в столбце **`city`** присутствуют пропуски. 

Т.к. другие ячейки в строках с пропущенными городами заполнены, то, чтобы не терять информацию, заполним пропуски некоторым обобщённым значением города, например строкой **`Unknown_city`**.

In [7]:
df.city.fillna('Unknown_city', inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
city    1000 non-null object
date    1000 non-null object
id      1000 non-null int64
dtypes: int64(1), object(2)
memory usage: 23.5+ KB


In [9]:
# Проверка дубликатов в данных
(df.duplicated() == True).sum()

0

In [10]:
# Сортировка данных по датам и последующая группировка по id
df_groupped = df.sort_values('date').groupby('id')

In [11]:
df.id.nunique()

291

In [12]:
# Проверка корректности группировки (совпадения числа различных id и числа полученных групп)
df_groupped.ngroups

291

In [13]:
df_groupped.size()[-10:]

id
289    2
290    1
291    5
292    6
293    1
294    5
295    4
296    4
297    5
299    2
dtype: int64

In [14]:
# Для каждого уникального id оставлена только одна строка с самой поздней датой
df_selected = df_groupped.nth(-1)
df_selected.head(15)

,city,date
id,,
0,"Kazan, Moscow, St-Petersburg",2014-10-06 20:58:34
1,"St-Petersburg, Moscow, Sochi, Kazan",2014-11-04 11:53:27
2,"Moscow, Kazan, St-Petersburg, Sochi",2014-07-02 02:04:42
3,"Moscow, Sochi",2014-10-31 13:06:28
4,"Kazan, Moscow, Sochi, St-Petersburg",2013-06-10 22:25:47
5,"Kazan, Sochi, Moscow",2013-03-23 11:45:54
6,"St-Petersburg, Sochi, Moscow",2014-02-06 23:17:13
7,"Sochi, St-Petersburg",2013-10-01 10:10:24
8,Kazan,2014-04-30 04:47:09


In [15]:
# Проверка корректности отбора данных: 
# количество отобранных записей должно совпадать с числом уникальных id
len(df_selected)

291

### а) сколько в каждом городе было уникальных id?

In [16]:
# В отобранных записях отделим города друг от друга
df_cities = df_selected['city'].str.split(expand=True).applymap(lambda x: x.rstrip(',') if x else x)
df_cities.head()

,0,1,2,3
id,,,,
0,Kazan,Moscow,St-Petersburg,None
1,St-Petersburg,Moscow,Sochi,Kazan
2,Moscow,Kazan,St-Petersburg,Sochi
3,Moscow,Sochi,None,None
4,Kazan,Moscow,Sochi,St-Petersburg


In [17]:
# Ответ на вопрос а)

df_cities.apply(lambda x: x.value_counts()).sum(axis=1)

Kazan            163.0
Moscow           163.0
Sochi            166.0
St-Petersburg    153.0
Unknown_city      52.0
dtype: float64

### b) сколько id содержится в каждом множестве городов?

In [18]:
df_selected.head(7)

,city,date
id,,
0,"Kazan, Moscow, St-Petersburg",2014-10-06 20:58:34
1,"St-Petersburg, Moscow, Sochi, Kazan",2014-11-04 11:53:27
2,"Moscow, Kazan, St-Petersburg, Sochi",2014-07-02 02:04:42
3,"Moscow, Sochi",2014-10-31 13:06:28
4,"Kazan, Moscow, Sochi, St-Petersburg",2013-06-10 22:25:47
5,"Kazan, Sochi, Moscow",2013-03-23 11:45:54
6,"St-Petersburg, Sochi, Moscow",2014-02-06 23:17:13


In [19]:
# Ответ на вопрос b)

df_selected['city'].str.replace(',', '').str.split().apply(lambda x: sorted(x)).astype(str).value_counts()

['Kazan', 'Moscow', 'Sochi', 'St-Petersburg']    70
['Unknown_city']                                 52
['Kazan', 'Moscow', 'Sochi']                     29
['Moscow', 'Sochi', 'St-Petersburg']             18
['Kazan', 'Moscow', 'St-Petersburg']             16
['St-Petersburg']                                15
['Sochi']                                        15
['Kazan']                                        12
['Kazan', 'Sochi', 'St-Petersburg']              11
['Kazan', 'Moscow']                              10
['Moscow', 'Sochi']                               8
['Sochi', 'St-Petersburg']                        8
['Kazan', 'St-Petersburg']                        8
['Moscow', 'St-Petersburg']                       7
['Kazan', 'Sochi']                                7
['Moscow']                                        5
Name: city, dtype: int64